# 멱등성을 보장하는 함수
- update_or_create()

In [8]:
user = User.objects.all()[0]
user, created = \
    User.objects.update_or_create(email=user.email, defaults={'name' : user.name})
print(user, created)

(0.015) SELECT "users"."id", "users"."email", "users"."name" FROM "users" LIMIT 1; args=(); alias=default
(0.115) SELECT "users"."id", "users"."email", "users"."name" FROM "users" WHERE "users"."email" = 'user1@django.com' LIMIT 21 FOR UPDATE; args=('user1@django.com',); alias=default
(0.020) UPDATE "users" SET "email" = 'user1@django.com', "name" = '유져1' WHERE "users"."id" = 1; args=('user1@django.com', '유져1', 1); alias=default


User object (1) False


## Model overriding
- save(update) 시에 count를 1증가시키는 로직을 추가
- save시에 update_fields를 지정하면 해당 내용만 업데이트 진행(오버헤드를 줄일 수 있다)

```python
class Account(models.Model):
    ....
    def save(self, *arg, **kwargs):
        self.people.like_count += 1
        self.people.save(
            update_fields=['like_count', 'updated_at'])
        super().save(*arg, **kwargs)
    ...
```

## Django model에서 추상클래스 사용
- Meta에 `abstract = True`를 하면 해당클래스는 추상클래스가 된다.

```python
from django.db import models

class TimeStampedModel(models.Model):
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)
    
    class Meta:
        abstract = True
        
class Board(TimeStampedModel):
    title = models.CharField(max_length=50)
    contents = models.TextField()
```

## Manager를 활용
- 자주쓰는 필터를 manager에 선언해서 사용
- 가독성이 좋고 반복작업을 하지 않아도됨

In [16]:
from django.db import models

class ActiveManager(models.Manager):
    def activated(self):
        return self.filter(is_activate=True)
    
    def deactivated(self):
        return self.filter(is_activate=False)